# Multiprocessing with Powerbox

There are two ways to parallelize the FFT calculations in `powerbox`. If you have `pyfftw` installed, you can take advantage of the multithreaded FFT computations that it offers (numpy does not support this), simply by setting `nthreads` to a number greater than one. However, if you would like to run many FFT's simultaneously, you may wish to parallelize on a higher level, i.e. run each FFT on a different process (using `multiprocessing` or `mpi` or similar). In this case, it is important that the underlying FFT library use only a single thread, or you will get VERY SLOW computation times because the two layers of threads don't communicate well. In this notebook, we show how to use both multiple threads via `pyfftw` and also multiple processes with either the `numpy` or `pyfftw` backends.

In [19]:
import powerbox as pb
pb.__version__

'0.7.4.dev28+gc727b58.d20240325'

First, let's define a simple `powerbox` operation to test. This function calculates a power spectrum on a random box of dimension $300^3$ and returns the computation time.

In [27]:
from powerbox import get_power
import numpy as np 
from time import time
from multiprocessing import Pool

def run_pb(arg1, arg2, **kwargs):
    t0 = time()
    out = get_power(arg1*arg2, (300,300,300),
                    bins = 50, **kwargs)                                      # default is nthreads = None which uses nthreads = number of available CPUs.
    return time() - t0

shape = (200,200,200) # Size of one chunk
arr = np.random.rand(np.prod(shape)).reshape(shape) # Random box on which to calculate the FFT

start = time()
ncalls = 8
all_times = []
for i in range(ncalls):
    all_times.append(run_pb(arr, i))
print('Total time:', np.round(np.sum(all_times),2),'s')

Total time: 7.74 s


## Multiprocessing with `pyFFTW` as a backend

We can keep using `pyFFTW` as a backend by setting the `nthreads` argument to 1.

In [28]:
nprocs = 8
ncalls = 8
start = time()
p = Pool(processes=nprocs)
for i in range(ncalls):
    p.apply_async(run_pb, args = (arr, i), kwds = {"nthreads" :1})
p.close()
p.join()
print('Total time:', np.round((time() - start),2),'s')

Total time: 11.42 s


## Multiprocessing with `numpy` as a backend

We can also just use the `numpy` FFT backend by setting `nthreads` to `False`.

In [29]:
nprocs = 8
ncalls = 8
start = time()
p = Pool(processes=nprocs)
for i in range(ncalls):
    p.apply_async(run_pb, args = (arr, i), kwds = {"nthreads" :False})
p.close()
p.join()
print('Total time:', np.round((time() - start),2),'s')

Total time: 15.21 s


The runtime is roughly the same whether we use `numpy` or single-threaded `pyFFTW`.